In [2]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings  # 경로 수정
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import chardet
import chromadb
from chromadb.config import Settings

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:

# .env 파일 로드
dotenv_path = os.path.join('/Users/passion1014/project/langchain/langserve-template/example', '..', '.env')
load_dotenv(dotenv_path)

# 1. a.txt 파일 로드
file_path = './norway.txt'
with open(file_path, 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    file_encoding = result['encoding']

with open(file_path, 'r', encoding=file_encoding) as file:
    text = file.read()


In [ ]:
# 2. 의미별로 chunk로 나누기
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=5)
chunks = text_splitter.split_text(text)
texts = text_splitter.create_documents([text])


# 3. chunk 데이터를 embedding
embedding_model = OpenAIEmbeddings()
new_client = chromadb.EphemeralClient()

chunk_embeddings = embedding_model.embed_documents(chunks)



In [4]:

client = chromadb.PersistentClient(path="/Users/passion1014/project/langchain/langserve-template/vectordb/chroma_db")  # 기존 데이터베이스 경로
logger.info(f"----- Chroma client connected: {client}")

# 기존 컬렉션 로드
db = Chroma(
    client=client,
    collection_name="openai_collection",
    embedding_function=OpenAIEmbeddings()
)
retriever = db.as_retriever(search_kwargs={"k": 1})  # 상위 5개 결과 반환
logger.info(f"----- Chroma collection loaded: {db}")

test_query = "와타나베가 점심 먹은곳은?"
test_results = retriever.get_relevant_documents(test_query)
logger.info(f"----- Test retrieval results: {test_results}")

collection = client.get_collection("openai_collection")
count = collection.count()
logger.info(f"----- Number of items in collection: {count}")
# ------------------------------------------------------------------



INFO:__main__:----- Chroma client connected: <chromadb.api.client.Client object at 0x13484c550>
INFO:chromadb.api.segment:Collection openai_collection is not created.
INFO:__main__:----- Chroma collection loaded: <langchain_community.vectorstores.chroma.Chroma object at 0x111f8b510>
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:----- Test retrieval results: [Document(page_content='여섯 시가 되자 우리  셋은 본관 식당으로 가서 저녁을 먹었다.   나오코와 나는 \n생선 구이에다  야채 샐러드, 조림, 그리고  밥과 된장국을 먹고,  레이코 여사는 \n마카로니 샐러드와 커피만을 들었다.  그리고 나서는 또 담배를 피웠다.\n  "나이가 들면 말이죠, 그다지 많이 먹지 않아도 괜찮도록 몸이 달라져요"\n  하며 그녀는 사뭇 설명조로 말했다.\n  식당에서 약 20여명이 테이블에 앉아 식사하는  동안에도, 몇 사람씩 들어오고 \n또 나갔다.  식당의 광경은 연령의 편차가  크다는 것을 제외하면 기숙사와 다를 \n바가 없었다.   한 가지 다르다면  누구나가 일정한 음량으로 말을  하고 있다는 \n점이었다.  크게 떠드는  사람도 없거나와 소곤거리는 사람도 없었다.  소리내어 \n웃거나 놀라거나,  손을 치켜들고 누굴 부른다거나  하는 사람은 아무도 없었다.   \n누구나가 같은 음량으로 조용하게 이야기하고 있었다.\n  그들은 몇 개의 테이블에  나누어 앉아 식사를 하고 있었다.   한 테이블에 세 \n사람, 많아야 다섯 사람이었다.  한 사람이 이야기하면 다른 사람들은 거

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import logging
import os

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# FAISS 인덱스 경로 설정
faiss_index_path = "/Users/passion1014/project/langchain/langserve-template/vectordb/mycollec"

# OpenAI 임베딩 초기화
embeddings = OpenAIEmbeddings()

# 안전하게 FAISS 인덱스 로드
try:
    db = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    logger.info(f"----- FAISS index loaded from: {faiss_index_path}")
except ValueError as e:
    logger.error(f"Error loading FAISS index: {e}")
    logger.info("Attempting to load index without embeddings...")
    # 임베딩 없이 로드 시도
    db = FAISS.load_local(faiss_index_path, allow_dangerous_deserialization=True)
    logger.info("FAISS index loaded without embeddings. Applying embeddings now.")
    db.embeddings = embeddings

# 검색기(Retriever) 설정
retriever = db.as_retriever(search_kwargs={"k": 1})  # 상위 1개 결과 반환
logger.info(f"----- FAISS retriever created")

# 테스트 쿼리 실행
test_query = "와타나베가 점심 먹은곳은?"
test_results = retriever.get_relevant_documents(test_query)
logger.info(f"----- Test retrieval results: {test_results}")

# FAISS 인덱스의 크기 확인
index_size = len(db.index_to_docstore_id)
logger.info(f"----- Number of items in FAISS index: {index_size}")

INFO:__main__:----- FAISS index loaded from: /Users/passion1014/project/langchain/langserve-template/vectordb/mycollec
INFO:__main__:----- FAISS retriever created
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:----- Test retrieval results: [Document(page_content='여섯 시가 되자 우리  셋은 본관 식당으로 가서 저녁을 먹었다.   나오코와 나는 \n생선 구이에다  야채 샐러드, 조림, 그리고  밥과 된장국을 먹고,  레이코 여사는 \n마카로니 샐러드와 커피만을 들었다.  그리고 나서는 또 담배를 피웠다.\n  "나이가 들면 말이죠, 그다지 많이 먹지 않아도 괜찮도록 몸이 달라져요"\n  하며 그녀는 사뭇 설명조로 말했다.\n  식당에서 약 20여명이 테이블에 앉아 식사하는  동안에도, 몇 사람씩 들어오고 \n또 나갔다.  식당의 광경은 연령의 편차가  크다는 것을 제외하면 기숙사와 다를 \n바가 없었다.   한 가지 다르다면  누구나가 일정한 음량으로 말을  하고 있다는 \n점이었다.  크게 떠드는  사람도 없거나와 소곤거리는 사람도 없었다.  소리내어 \n웃거나 놀라거나,  손을 치켜들고 누굴 부른다거나  하는 사람은 아무도 없었다.   \n누구나가 같은 음량으로 조용하게 이야기하고 있었다.\n  그들은 몇 개의 테이블에  나누어 앉아 식사를 하고 있었다.   한 테이블에 세 \n사람, 많아야 다섯 사람이었다.  한 사람이 이야기하면 다른 사람들은 거기에 귀\n를 기울이며 응,  응, 하면서 수긍을 하고,  그 사람의 말이 끝나면  다른 사람이 \n거기에 대해서 얼마 동안 말을 하곤 했다.\n  무슨 이야기를 하고 있는지는 알 수가 없었지만,  그들의 대화는 내가

In [5]:

import tiktoken
from typing import List
from langchain.schema import Document

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """주어진 문자열의 토큰 수를 반환합니다."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def calculate_tokens_for_documents(docs: List[Document]) -> int:
    """Document 객체 리스트의 총 토큰 수를 계산합니다."""
    total_tokens = 0
    for doc in docs:
        total_tokens += num_tokens_from_string(doc.page_content)
    return total_tokens

# test_results의 토큰 수 계산
total_tokens = calculate_tokens_for_documents(test_results)

logger.info(f"----- Total tokens in test_results: {total_tokens}")


INFO:__main__:----- Total tokens in test_results: 6753


In [ ]:

def calculate_tokens_for_documents(docs: List[Document]) -> int:
    """Document 객체 리스트의 총 토큰 수를 계산합니다."""
    total_tokens = 0
    for doc in docs:
        total_tokens += num_tokens_from_string(doc.page_content)
    return total_tokens

# test_results의 토큰 수 계산
total_tokens = calculate_tokens_for_documents(test_results)

logger.info(f"----- Total tokens in test_results: {total_tokens}")


In [3]:
# load it into Chroma
# db = Chroma.from_documents(chunks, embedding_model, client=new_client, collection_name="openai_collection")
db = Chroma.from_documents(documents=texts, embedding=embedding_model)




NameError: name 'texts' is not defined

In [19]:

query = "이 책의 작가는 누구야?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

상실의 시대
   지은이: 무라카미하루키
   출판사: 문학사상사
   

   상실의 시대
   제 1장 나를 꼭 기억해 주었으면 해요
